# MAT 328 - Final Project
### Tony Lu

---

This project uses the [Motor Vehicle Collision](https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95) dataset offered by [NYC Open Data](http://opendata.cityofnewyork.us/).

All data and values are dated March 2021 - the most recent date this notebook was executed. Executing this notebook at a later date may yield different results compared to what is already rendered in this notebook.

# Project Setup

### Import the necessary libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
sns.set_theme(style='darkgrid')

### Download and load the data

In [2]:
# This might take awhile. It has 1.7 million rows after all.
raw_data = pd.read_csv('https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv?accessType=DOWNLOAD')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df = raw_data.copy()

### Show the data

In [4]:
df.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,10/19/2018,0:00,NaN,NaN,40.834060,-73.944885,"(40.83406, -73.944885)",BROADWAY,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Oversized Vehicle,Driver Inattention/Distraction,NaN,NaN,NaN,4004397,Sedan,Chassis Cab,NaN,NaN,NaN
1,10/06/2018,16:39,BRONX,10456,40.828194,-73.903860,"(40.828194, -73.90386)",BOSTON ROAD,EAST 167 STREET,NaN,0.0,0.0,0,0,0,0,0,0,Failure to Yield Right-of-Way,Unspecified,NaN,NaN,NaN,3994805,Sedan,Sedan,NaN,NaN,NaN
2,10/14/2018,12:10,QUEENS,11411,40.692230,-73.727520,"(40.69223, -73.72752)",NaN,NaN,116-28 234 STREET,1.0,0.0,0,0,0,0,1,0,Following Too Closely,Unspecified,Unspecified,NaN,NaN,4001451,Sedan,Sedan,Sedan,NaN,NaN
3,10/20/2018,19:30,NaN,NaN,40.708324,-73.843140,"(40.708324, -73.84314)",JACKIE ROBINSON PKWY,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,4004650,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN
4,10/13/2018,12:30,QUEENS,11355,40.756480,-73.831360,"(40.75648, -73.83136)",FRAME PLACE,41 ROAD,NaN,0.0,0.0,0,0,0,0,0,0,Passing Too Closely,Unspecified,NaN,NaN,NaN,3999269,Station Wagon/Sport Utility Vehicle,FLATB,NaN,NaN,NaN


In [5]:
# Show size of the data
print(f'Rows: {df.shape[0]} | Columns: {df.shape[1]}')

Rows: 1759556 | Columns: 29


In [6]:
df.describe(include='all')

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
count,1759556,1759556,1219220,1219009.0,1.550996e+06,1.550996e+06,1550996,1405116,1141381,262072,1.759538e+06,1.759525e+06,1.759556e+06,1.759556e+06,1.759556e+06,1.759556e+06,1.759556e+06,1.759556e+06,1754613,1510702,117778,25315,6609,1.759556e+06,1750309,1468911,114591,24603,6435
unique,3167,1440,5,421.0,NaN,NaN,228157,12095,18866,160483,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,61,50,39,29,NaN,1196,1291,182,74,53
top,01/21/2014,16:00,BROOKLYN,11207.0,NaN,NaN,"(0.0, 0.0)",BROADWAY,3 AVENUE,772 EDGEWATER ROAD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,NaN,Sedan,PASSENGER VEHICLE,Sedan,Sedan,Sedan
freq,1161,25440,382027,15001.0,NaN,NaN,1280,17196,9843,402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,630355,1270148,109597,23864,6224,NaN,429376,318607,30538,7028,1899
mean,NaN,NaN,NaN,NaN,4.069056e+01,-7.386784e+01,NaN,NaN,NaN,NaN,2.726687e-01,1.255452e-03,5.147094e-02,6.603939e-04,2.280973e-02,9.547863e-05,1.982233e-01,4.984212e-04,NaN,NaN,NaN,NaN,NaN,2.909304e+06,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,1.172107e+00,2.375114e+00,NaN,NaN,NaN,NaN,6.679621e-01,3.746889e-02,2.333875e-01,2.623689e-02,1.508869e-01,9.828846e-03,6.297285e-01,2.441158e-02,NaN,NaN,NaN,NaN,NaN,1.502984e+06,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,0.000000e+00,-2.013600e+02,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,2.200000e+01,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,4.066862e+01,-7.397605e+01,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,2.865733e+06,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,4.072218e+01,-7.392844e+01,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,3.515936e+06,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,4.076881e+01,-7.386667e+01,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,3.956062e+06,NaN,NaN,NaN,NaN,NaN


### Data Handling
- Clean up unneeded data
- Make sure the correct dtypes are being used
- Handle missing data

#### Unneeded data

In [7]:
# We don't need COLLISION_ID as its more of a unique identifier for internal use
# LOCATION is just a combination of LONGITUDE and LATITUDE, having either is fine
df.drop(['COLLISION_ID', 'LOCATION'], axis=1, inplace=True)

#### Check and handle missing values

In [8]:
# Check which columns have missing values
nulls = df.isnull().sum()
nulls[nulls > 0]

BOROUGH                           540336
ZIP CODE                          540547
LATITUDE                          208560
LONGITUDE                         208560
ON STREET NAME                    354440
CROSS STREET NAME                 618175
OFF STREET NAME                  1497484
NUMBER OF PERSONS INJURED             18
NUMBER OF PERSONS KILLED              31
CONTRIBUTING FACTOR VEHICLE 1       4943
CONTRIBUTING FACTOR VEHICLE 2     248854
CONTRIBUTING FACTOR VEHICLE 3    1641778
CONTRIBUTING FACTOR VEHICLE 4    1734241
CONTRIBUTING FACTOR VEHICLE 5    1752947
VEHICLE TYPE CODE 1                 9247
VEHICLE TYPE CODE 2               290645
VEHICLE TYPE CODE 3              1644965
VEHICLE TYPE CODE 4              1734953
VEHICLE TYPE CODE 5              1753121
dtype: int64

`CONTRIBUTING FACTOR VEHICLE` and `VEHICLE TYPE CODE` are used to identify the vehicle involved in the collision as well as the vehicle's role in the collision. 

`CONTRIBUTING FACTOR VEHICLE` and `VEHICLE TYPE CODE` can be kept as null. For a particular collision with 2 vehicles, The data should include information for these two vehicles but vehicles 3, 4, and 5 should be empty - it would make sense that values are missing from these columns.

In [9]:
# Drop missing values
# OFF STREET NAME has too many missing values for us to take care of.
# With over 1.4 million missing values, it's over 80% of the data.
df.drop('OFF STREET NAME', axis=1, inplace=True)

# The data serves us no purpose if the geo-coordinates are missing.
# The coords can be used to retrieve info about borough, street, ZIP code
df.dropna(how='any', subset=['LATITUDE', 'LONGITUDE'], inplace=True)

In [10]:
# Impute missing values

# For "NUMBER OF PERSONS INJURED" and "NUMBER OF PERSONS KILLED",
# There is only a small amount of values missing which makes it easy for us to impute it

# Impute "NUMBER OF PERSONS INJURED" via mean
injured_mean = df['NUMBER OF PERSONS INJURED'].mean()
df['NUMBER OF PERSONS INJURED'].replace(np.nan, injured_mean, inplace=True)

# Impute "NUMBER OF PERSONS KILLED" via mean
killed_mean = df['NUMBER OF PERSONS KILLED'].mean()
df['NUMBER OF PERSONS KILLED'].replace(np.nan, killed_mean, inplace=True)

The next few columns to worry about are `ZIP CODE`, `BOROUGH`, and `ON STREET NAME`. If we check the number of records with values missing in any of the 3 columns, we will see that there is about 600,000 values missing in total. Or about 35% of the total data - which is a big chunk of the total data but not enough to leave a huge impact if we drop them. By dropping them, we'll still have about 1 million left.

In [11]:
filter = df[['ZIP CODE', 'BOROUGH', 'ON STREET NAME']].isnull().any(1)
df[filter].shape

(589558, 26)

In [12]:
df.dropna(how='any', subset=['ZIP CODE', 'BOROUGH', 'ON STREET NAME'], inplace=True)
df.shape

(961438, 26)

#### Check dtypes

In [13]:
df.dtypes

CRASH DATE                        object
CRASH TIME                        object
BOROUGH                           object
ZIP CODE                          object
LATITUDE                         float64
LONGITUDE                        float64
ON STREET NAME                    object
CROSS STREET NAME                 object
NUMBER OF PERSONS INJURED        float64
NUMBER OF PERSONS KILLED         float64
NUMBER OF PEDESTRIANS INJURED      int64
NUMBER OF PEDESTRIANS KILLED       int64
NUMBER OF CYCLIST INJURED          int64
NUMBER OF CYCLIST KILLED           int64
NUMBER OF MOTORIST INJURED         int64
NUMBER OF MOTORIST KILLED          int64
CONTRIBUTING FACTOR VEHICLE 1     object
CONTRIBUTING FACTOR VEHICLE 2     object
CONTRIBUTING FACTOR VEHICLE 3     object
CONTRIBUTING FACTOR VEHICLE 4     object
CONTRIBUTING FACTOR VEHICLE 5     object
VEHICLE TYPE CODE 1               object
VEHICLE TYPE CODE 2               object
VEHICLE TYPE CODE 3               object
VEHICLE TYPE COD

In [14]:
# Fix the dtypes

df['CRASH DATE'] = pd.to_datetime(df['CRASH DATE'])
df['CRASH TIME'] = pd.to_datetime(df['CRASH TIME'])

# These should be ints, not floats - can't have half a person dead
df['NUMBER OF PERSONS INJURED'] = df['NUMBER OF PERSONS INJURED'].astype(int)
df['NUMBER OF PERSONS KILLED'] = df['NUMBER OF PERSONS KILLED'].astype(int)

#### Miscellaneous Cleanup

In [15]:
# Convert everything to uppercase to be consistent
# This might take a few seconds or minutes depending on system
# Took about ~10 seconds via Google Colab
df = df.applymap(lambda s:s.upper() if type(s) == str else s)
df.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,ON STREET NAME,CROSS STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
1,2018-10-06,2021-03-08 16:39:00,BRONX,10456,40.828194,-73.903860,BOSTON ROAD,EAST 167 STREET,0,0,0,0,0,0,0,0,FAILURE TO YIELD RIGHT-OF-WAY,UNSPECIFIED,NaN,NaN,NaN,SEDAN,SEDAN,NaN,NaN,NaN
4,2018-10-13,2021-03-08 12:30:00,QUEENS,11355,40.756480,-73.831360,FRAME PLACE,41 ROAD,0,0,0,0,0,0,0,0,PASSING TOO CLOSELY,UNSPECIFIED,NaN,NaN,NaN,STATION WAGON/SPORT UTILITY VEHICLE,FLATB,NaN,NaN,NaN
5,2018-10-06,2021-03-08 21:45:00,QUEENS,11372,40.746810,-73.891760,BROADWAY,ROOSEVELT AVENUE,0,0,0,0,0,0,0,0,DRIVER INATTENTION/DISTRACTION,UNSPECIFIED,NaN,NaN,NaN,STATION WAGON/SPORT UTILITY VEHICLE,SEDAN,NaN,NaN,NaN
13,2018-10-16,2021-03-08 14:30:00,MANHATTAN,10001,40.754616,-73.997116,WEST 35 STREET,DYER AVENUE,0,0,0,0,0,0,0,0,TRAFFIC CONTROL DISREGARDED,TRAFFIC CONTROL DISREGARDED,NaN,NaN,NaN,STATION WAGON/SPORT UTILITY VEHICLE,STATION WAGON/SPORT UTILITY VEHICLE,NaN,NaN,NaN
16,2018-10-18,2021-03-08 22:30:00,QUEENS,11364,40.745790,-73.768776,HORACE HARDING EXPRESSWAY,208 STREET,0,0,0,0,0,0,0,0,FAILURE TO YIELD RIGHT-OF-WAY,UNSPECIFIED,NaN,NaN,NaN,SEDAN,STATION WAGON/SPORT UTILITY VEHICLE,NaN,NaN,NaN
